In [1]:
import pandas as pd
import sklearn

from sklearn.isotonic import IsotonicRegression

In [2]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import BoxAnnotation, Arrow, NormalHead, Label
from bokeh import palettes
from bokeh.resources import CDN
from bokeh.embed import file_html
output_notebook()

lightblue, darkblue, lightred, darkred, lightgreen, darkgreen = palettes.Paired[6]

Loading BokehJS ...

In [3]:
def fit_isotonic_regression(ts, rhs_portion):
    rhs = ts.tail(int(len(ts) * rhs_portion))
    
    ir = IsotonicRegression(y_max=rhs.mean())
    
    return pd.Series(ir.fit_transform(ts.index, ts.values), index=ts.index)

def find_equilibrium(ts, rhs_portion):
    """Find the point where the signal becomes equilibrated
    
    Parameters
    ----------
    ts : pd.Series
      time series to find equilibrium for
    rhs_portion : float
    
    Returns
    -------
    equilibrium : float
      index of where the time series reaches equilibrium
    """
    iso = fit_isotonic_regression(ts, rhs_portion)
    
    eq = iso[iso == iso.iloc[-1]].index[0]
    
    return eq

In [4]:
dlm_ts = pd.read_csv('dlm_ts.csv', header=None, squeeze=True, index_col=0)

ts = dlm_ts.iloc[:25000]

In [5]:
def plot_bokeh_isotronic(ts, rhs_portion):
    iso_ts = fit_isotonic_regression(ts, rhs_portion)
    
    n_rhs = int(len(ts) * rhs_portion)
    n_lhs = len(ts) - n_rhs
    lhs = ts.head(n_lhs)
    rhs = ts.tail(n_rhs)
    rhs_mean = rhs.mean()
    print("Y mean value: {}".format(rhs_mean))

    first_x = iso_ts[iso_ts == iso_ts.iloc[-1]].index[0]
    print("First x value: {}".format(first_x))
    
    p1 = figure(plot_width=750, plot_height=250, output_backend='webgl')
    eq_box = BoxAnnotation(right=first_x, fill_alpha=0.25,
                           fill_color=lightred, line_color=darkred)
    p1.add_layout(eq_box)
    p1.line(lhs.index, lhs.values, line_color=lightblue)
    p1.line(rhs.index, rhs.values, line_color=lightgreen, line_alpha=0.5)
    
    # plot rhs used for mean estimation
    p1.line([rhs.index[0], rhs.index[-1]], [rhs_mean] * 2,
            color=darkgreen, line_dash='dotted')
    # plot fitted data
    lhs_iso = iso_ts[:rhs.index[0]]
    p1.line(lhs_iso.index, lhs_iso.values, line_color=darkblue)
    
    p1.xaxis.axis_label = 'Simulation duration'
    p1.yaxis.axis_label = 'Adsorbed amount'
    
    html = file_html(p1, CDN, 'equil')
    with open('equil.html', 'w') as f:
        for line in html.split('\n'):
            f.write(line.lstrip() + '\n')
    show(p1)
    
plot_bokeh_isotronic(ts, 0.25)

Y mean value: 135.5748
First x value: 2249000.0


In [6]:
plot = figure(plot_width=750, plot_height=250,
             output_backend='webgl')

plot.line(ts.index, ts.values, color=lightblue)

sampling_zone = Arrow(end=NormalHead(line_color=darkblue,
                                     fill_color=darkblue),
                      x_start=3.0e6, x_end=10.0e6,
                      y_start=135, y_end=135,
                      line_color=darkblue,
                      line_width=10.0,
                     )
sample_text = Label(x=5e6, y=110, text='Sampling')
plot.add_layout(sampling_zone)
plot.add_layout(sample_text)

equil_zone = Arrow(end=NormalHead(line_color=darkred,
                                  fill_color=darkred,
                                  ),
                  x_start=0, x_end=1.5e6,
                  y_start=0, y_end=125,
                  line_color=darkred,
                  line_width=10.0,
                  )
equil_text = Label(x=1.25e6, y=60, text='Reaching equilibrium')
plot.add_layout(equil_zone)
plot.add_layout(equil_text)

show(plot)

In [7]:
plot = figure(plot_width=750, plot_height=250)

plot.line(ts.index, ts.values, line_color=lightblue)

ir = IsotonicRegression()
y_ = ir.fit_transform(ts.index, ts.values)

plot.line(ts.index, y_, line_color=darkblue)

show(plot)